# Working with data on PySpark

1. [Spark DataFrames](#section1)
2. [Joining two DataFrames](#section2)
3. [Using SQL to query DataFrames](#section3)
4. [Writing out results](#section4)
5. [Data bucketing and partitioning](#section5)

From SparkSQL doccumentation: https://spark.apache.org/docs/2.3.0/sql-programming-guide.html

In [1]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *

S3_BUCKETNAME = ''
S3_PREFIX = 'pysparklab'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
24,application_1616621397322_0025,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<a id="section1"></a>
## 1. Working with Spark DataFrames
A DataFrame is a <i>Dataset</i> organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood.

DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R.

In the simplest form, the default data source (parquet unless otherwise configured by spark.sql.sources.default) will be used for all operations.

### 1.1 Create a DataFrame from a file

In [2]:
path = "s3://{}/{}/users.parquet".format(S3_BUCKETNAME,S3_PREFIX)
df = spark.read.load(path)
type(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

<br><br>
### 1.2 View the DataFrame

In [3]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+

<br><br>
### 1.3 Use custom options with the DataFrame

You can also manually specify the data source that will be used along with any extra options that you would like to pass to the data source. Data sources are specified by their fully qualified name (i.e., org.apache.spark.sql.parquet), but for built-in sources you can also use their short names (json, parquet, jdbc, orc, libsvm, csv, text). 

Take the following nyt2.json file as an example:
```json
{
    "_id": {
        "oid": "5b4aa4ead3089013507db18b"
    },
    "bestsellers_date": {
        "date": {
            "numberLong": "1211587200000"
        }
    },
    "published_date": {
        "date": {
            "$numberLong": "1212883200000"
        }
    },
    "amazon_product_url": "http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20",
    "author": "Dean R Koontz",
    "description": "Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.",
    "price": {
        "numberInt": "27"
    },
    "publisher": "Bantam",
    "title": "ODD HOURS",
    "rank": {
        "numberInt": "1"
    },
    "rank_last_week": {
        "numberInt": "0"
    },
    "weeks_on_list": {
        "numberInt": "1"
    }
}
```

Notice the <b>format="json"</b> options specified on the read.load() function

In [4]:
path = "s3://{}/{}/nyt2.json".format(S3_BUCKETNAME,S3_PREFIX)
df = spark.read.load(path, format="json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br>
### 1.4 Show the DataFrame schema

In [5]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- amazon_product_url: string (nullable = true)
 |-- author: string (nullable = true)
 |-- bestsellers_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- $numberDouble: string (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- published_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- rank: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- rank_last_week: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- title: string (nullable = true)
 |-- weeks_on_list: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)

<br><br>
### 1.5 Access the DataFrame
In Python it’s possible to access a DataFrame’s columns either by attribute (df.age) or by indexing (df['age']). 

In [6]:
df.author

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Column<b'author'>

In [7]:
df['author']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Column<b'author'>

<br><br>
### 1.6 Using select()
You can also select columns by using the <b>select()</b> function

In [8]:
df.select("author", "title", "rank", "price").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----+--------+
|              author|               title|rank|   price|
+--------------------+--------------------+----+--------+
|       Dean R Koontz|           ODD HOURS| [1]|  [, 27]|
|     Stephenie Meyer|            THE HOST| [2]|[25.99,]|
|        Emily Giffin|LOVE THE ONE YOU'...| [3]|[24.95,]|
|   Patricia Cornwell|           THE FRONT| [4]|[22.95,]|
|     Chuck Palahniuk|               SNUFF| [5]|[24.95,]|
|James Patterson a...|SUNDAYS AT TIFFANY’S| [6]|[24.99,]|
|       John Sandford|        PHANTOM PREY| [7]|[26.95,]|
|       Jimmy Buffett|          SWINE NOT?| [8]|[21.99,]|
|    Elizabeth George|     CARELESS IN RED| [9]|[27.95,]|
|      David Baldacci|     THE WHOLE TRUTH|[10]|[26.99,]|
+--------------------+--------------------+----+--------+
only showing top 10 rows

<br><br>
### 1.7 Filtering records

Using <b>filter()</b> to select titles where the author is one of the list ["John Sandford", "Emily Giffin"]

In [9]:
df.filter(df.author.isin("John Sandford", "Emily Giffin")).select("author", "title").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+
|       author|               title|
+-------------+--------------------+
| Emily Giffin|LOVE THE ONE YOU'...|
|John Sandford|        PHANTOM PREY|
| Emily Giffin|LOVE THE ONE YOU'...|
|John Sandford|        PHANTOM PREY|
| Emily Giffin|LOVE THE ONE YOU'...|
+-------------+--------------------+
only showing top 5 rows

<br><br>
Using <b>when()</b> to find titles that contains LOVE in the name. Notice we are using a new function here called <b>col()</b> instead of using df.title

In [10]:
df.select("title", when(col('title').like('LOVE%'), 1).otherwise(0)).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------------------------------+
|               title|CASE WHEN title LIKE LOVE% THEN 1 ELSE 0 END|
+--------------------+--------------------------------------------+
|           ODD HOURS|                                           0|
|            THE HOST|                                           0|
|LOVE THE ONE YOU'...|                                           1|
|           THE FRONT|                                           0|
|               SNUFF|                                           0|
|SUNDAYS AT TIFFANY’S|                                           0|
|        PHANTOM PREY|                                           0|
|          SWINE NOT?|                                           0|
|     CARELESS IN RED|                                           0|
|     THE WHOLE TRUTH|                                           0|
+--------------------+--------------------------------------------+
only showing top 10 rows

In [11]:
df = df.withColumn('new_column', lit('This is a new column'))
df.select("new_column").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|          new_column|
+--------------------+
|This is a new column|
|This is a new column|
|This is a new column|
|This is a new column|
|This is a new column|
+--------------------+
only showing top 5 rows

<br><br>
### 1.8 Managing Columns: create a new column using <b>withColumn()</b> function

In [12]:
df = df.withColumn('priceFloat', df.price['$numberDouble'].cast(FloatType()))
df.select("priceFloat","price").printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- priceFloat: float (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- $numberDouble: string (nullable = true)
 |    |-- $numberInt: string (nullable = true)

<br><br>
### 1.9 Group the DataFrame and calculate average

In [13]:
df = df.groupBy("author").avg('priceFloat')
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+
|           author|   avg(priceFloat)|
+-----------------+------------------+
|       James Frey|26.950000762939453|
| Elin Hilderbrand|26.444545225663617|
|Sharon Kay Penman|28.950000762939453|
|      Lisa Genova|              null|
|     Will Allison|              null|
|Patricia Cornwell|27.679730492669183|
|    Laurie R King|              null|
|       Tea Obreht|              null|
|     Sarah Dunant|              null|
|     Tim Johnston|              null|
+-----------------+------------------+
only showing top 10 rows

<br><br>
### 1.10 Drop null records and sort by price Descending

In [14]:
df = df.na.drop().sort(col('avg(priceFloat)').desc())
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|              author|   avg(priceFloat)|
+--------------------+------------------+
|Robert Jordan and...| 32.21222284105089|
|     Haruki Murakami|              30.5|
|           Dan Brown|29.950000762939453|
|     Neal Stephenson|29.950000762939453|
|    Patrick Rothfuss|29.950000762939453|
|          Wally Lamb|29.950000762939453|
|          Pat Conroy|29.950000762939453|
| Stephen R Donaldson|29.950000762939453|
|      Terry Goodkind|29.480000019073486|
|Michael Crichton ...|28.989999771118164|
+--------------------+------------------+
only showing top 10 rows

<br><br>
### 1.11 DataFrame stadistics
Count records using <b>count()</b>

In [15]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

324

<br><br>
Using <b>describe()</b> function

In [16]:
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------------------+
|summary|              author|   avg(priceFloat)|
+-------+--------------------+------------------+
|  count|                 324|               324|
|   mean|                null| 26.46742033771837|
| stddev|                null|1.5429135925252075|
|    min|          Ace Atkins|19.987142699105398|
|    max|edited  Charlaine...| 32.21222284105089|
+-------+--------------------+------------------+

<br><br>
<a id="section2"></a>
## 2. Joining two DataFrames

### 2.1 Read and explore the States CSV
Notice the <b>read.option()<b> and <b>read.csv()<b> methods

In [17]:
statesDf = spark.read.option("header", "true").csv('s3://databrew-public-datasets-us-east-1/states.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
statesDf.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: string (nullable = true)
 |-- assembly_session: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- state_name: string (nullable = true)
 |-- all_votes: string (nullable = true)
 |-- yes_votes: string (nullable = true)
 |-- no_votes: string (nullable = true)
 |-- abstain: string (nullable = true)
 |-- idealpoint_estimate: string (nullable = true)
 |-- affinityscore_usa: string (nullable = true)
 |-- affinityscore_russia: string (nullable = true)
 |-- affinityscore_china: string (nullable = true)
 |-- affinityscore_india: string (nullable = true)
 |-- affinityscore_brazil: string (nullable = true)
 |-- affinityscore_israel: string (nullable = true)

<br><br>
Notice we are using <b>take()</b> instead of show(), which is used only to visualize the DataFrame. With take() you can use the output to create a new DataFrame

In [19]:
statesDf.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(year='1946', assembly_session='1', state_code='2', state_name='United States of America', all_votes='42', yes_votes='25', no_votes='15', abstain='2', idealpoint_estimate='1.7377', affinityscore_usa='1', affinityscore_russia='0.2143', affinityscore_china=None, affinityscore_india='0.4762', affinityscore_brazil='0.6429', affinityscore_israel=None)]

In [20]:
statesDf.select('year', 'state_code', 'state_name', 'all_votes', 'assembly_session').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+--------------------+---------+----------------+
|year|state_code|          state_name|all_votes|assembly_session|
+----+----------+--------------------+---------+----------------+
|1946|         2|United States of ...|       42|               1|
|1947|         2|United States of ...|       38|               2|
|1948|         2|United States of ...|      103|               3|
|1949|         2|United States of ...|       63|               4|
|1950|         2|United States of ...|       53|               5|
|1951|         2|United States of ...|       25|               6|
|1952|         2|United States of ...|       49|               7|
|1953|         2|United States of ...|       25|               8|
|1954|         2|United States of ...|       30|               9|
|1955|         2|United States of ...|       27|              10|
+----+----------+--------------------+---------+----------------+
only showing top 10 rows

<br><br>
### 2.2 Read and explore the Votes CSV

In [21]:
votesDf = spark.read.option("header", "true").csv('s3://databrew-public-datasets-us-east-1/votes.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
votesDf.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- assembly_session: string (nullable = true)
 |-- vote_id: string (nullable = true)
 |-- resolution: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- state_name: string (nullable = true)
 |-- member: string (nullable = true)
 |-- vote: string (nullable = true)

In [23]:
votesDf.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------+----------+----------+--------------------+------+----+
|assembly_session|vote_id|resolution|state_code|          state_name|member|vote|
+----------------+-------+----------+----------+--------------------+------+----+
|               1|      3|    R/1/66|         2|United States of ...|     1|   1|
|               1|      3|    R/1/66|        20|              Canada|     1|   3|
|               1|      3|    R/1/66|        31|             Bahamas|     0|   9|
|               1|      3|    R/1/66|        40|                Cuba|     1|   1|
|               1|      3|    R/1/66|        41|               Haiti|     1|   1|
|               1|      3|    R/1/66|        42|  Dominican Republic|     1|   1|
|               1|      3|    R/1/66|        51|             Jamaica|     0|   9|
|               1|      3|    R/1/66|        52| Trinidad and Tobago|     0|   9|
|               1|      3|    R/1/66|        53|            Barbados|     0|   9|
|               

<br><br>
### 2.3 Read and explore the Resolution CSV

In [24]:
resolutionDf = spark.read.option("header", "true").csv('s3://databrew-public-datasets-us-east-1/resolution.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
resolutionDf.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- assembly_session: string (nullable = true)
 |-- vote_id: string (nullable = true)
 |-- resolution: string (nullable = true)
 |-- amendment: string (nullable = true)
 |-- vote_date: string (nullable = true)
 |-- significant_vote: string (nullable = true)
 |-- yes_votes: string (nullable = true)
 |-- no_votes: string (nullable = true)
 |-- abstain: string (nullable = true)
 |-- colonization: string (nullable = true)
 |-- human_rights: string (nullable = true)
 |-- israel_palestine: string (nullable = true)
 |-- disarmament: string (nullable = true)
 |-- nuclear_weapons: string (nullable = true)
 |-- economic_development: string (nullable = true)

<br><br>
### 2.4 Prior to join, we select the columns needed

In [26]:
resolutionDf = resolutionDf.select('assembly_session', 'disarmament', 'human_rights', 'economic_development', \
    'yes_votes', 'no_votes', 'abstain')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
votesDf = votesDf.select('state_name', 'assembly_session')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br>
### 2.5 Next, we join the Resolution and Votes DataFrame by the column assemply_session

In [28]:
joined_dataframe = resolutionDf.join(votesDf, resolutionDf.assembly_session == votesDf.assembly_session)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br>
### 2.6 Then, we scope down the analysis to the country of interest

In [29]:
joined_dataframe = joined_dataframe.filter(joined_dataframe.state_name == 'Chile')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br>
### 2.7 Finally, let's take a look into the joined DataFrame

In [30]:
joined_dataframe.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- assembly_session: string (nullable = true)
 |-- disarmament: string (nullable = true)
 |-- human_rights: string (nullable = true)
 |-- economic_development: string (nullable = true)
 |-- yes_votes: string (nullable = true)
 |-- no_votes: string (nullable = true)
 |-- abstain: string (nullable = true)
 |-- state_name: string (nullable = true)
 |-- assembly_session: string (nullable = true)

In [31]:
joined_dataframe.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+------------+--------------------+---------+--------+-------+----------+----------------+
|assembly_session|disarmament|human_rights|economic_development|yes_votes|no_votes|abstain|state_name|assembly_session|
+----------------+-----------+------------+--------------------+---------+--------+-------+----------+----------------+
|               1|          0|           0|                   0|       14|      35|      4|     Chile|               1|
|               1|          0|           0|                   0|       30|       5|     18|     Chile|               1|
|               1|          0|           0|                   1|       34|      11|      8|     Chile|               1|
|               1|          0|           0|                   1|       25|      22|      6|     Chile|               1|
|               1|          0|           0|                   1|       15|      28|     10|     Chile|               1|
|               1|          0|          

<br><br>
<a id="section3"></a>
## 3. Using SQL to query DataFrames

### 3.1 We use <b>registerTempTable()</b> function to register a DataFrame as a Temporary Table in the SQLContext

In [32]:
joined_dataframe.registerTempTable("unvotes")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br>
### 3.2 Now we can create our SQL queries and run them in a simple way
Notice the <b>%%sql</b> IPython magic command. This query will run under the SQL context

In [33]:
%%sql
select 'human_rights', sum(yes_votes), sum(no_votes), sum(abstain) from unvotes where human_rights = 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<br><br>
### 3.3 Now lets get the sum of yes, no and abstain votes for human_rights, economic_development and disarmament using <b>spark.sql()</b>

In [34]:
query = query = "select 'human_rights', sum(yes_votes), sum(no_votes), sum(abstain) from unvotes where human_rights = 1"
human_rights = spark.sql(query)
human_rights.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------------------+-----------------------------+----------------------------+
|human_rights|sum(CAST(yes_votes AS DOUBLE))|sum(CAST(no_votes AS DOUBLE))|sum(CAST(abstain AS DOUBLE))|
+------------+------------------------------+-----------------------------+----------------------------+
|human_rights|                   1.0118779E7|                    1003432.0|                   2054058.0|
+------------+------------------------------+-----------------------------+----------------------------+

In [35]:
query = "select 'economic_development', sum(yes_votes), sum(no_votes), sum(abstain) from unvotes where economic_development = 1"
economic_development = spark.sql(query)
economic_development.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------------+-----------------------------+----------------------------+
|economic_development|sum(CAST(yes_votes AS DOUBLE))|sum(CAST(no_votes AS DOUBLE))|sum(CAST(abstain AS DOUBLE))|
+--------------------+------------------------------+-----------------------------+----------------------------+
|economic_development|                     5611881.0|                     319123.0|                    774589.0|
+--------------------+------------------------------+-----------------------------+----------------------------+

In [36]:
query = "select 'disarmament', sum(yes_votes), sum(no_votes), sum(abstain) from unvotes where disarmament = 1"
disarmament = spark.sql(query)
disarmament.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------------------+-----------------------------+----------------------------+
|disarmament|sum(CAST(yes_votes AS DOUBLE))|sum(CAST(no_votes AS DOUBLE))|sum(CAST(abstain AS DOUBLE))|
+-----------+------------------------------+-----------------------------+----------------------------+
|disarmament|                   1.1687243E7|                     732267.0|                   1662575.0|
+-----------+------------------------------+-----------------------------+----------------------------+

<br><br>
### 3.4 Then, we concatenate the three DataFrames using <b>union()</b>

In [37]:
totals = human_rights.union(economic_development).union(disarmament)
totals = totals.withColumnRenamed('human_rights', 'topic')
totals = totals.withColumnRenamed('sum(CAST(yes_votes AS DOUBLE))', 'yes')
totals = totals.withColumnRenamed('sum(CAST(no_votes AS DOUBLE))', 'no')
totals = totals.withColumnRenamed('sum(CAST(abstain AS DOUBLE))', 'abstain')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
totals.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+---------+---------+
|               topic|        yes|       no|  abstain|
+--------------------+-----------+---------+---------+
|        human_rights|1.0118779E7|1003432.0|2054058.0|
|economic_development|  5611881.0| 319123.0| 774589.0|
|         disarmament|1.1687243E7| 732267.0|1662575.0|
+--------------------+-----------+---------+---------+

In [39]:
totals.registerTempTable("totals")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br>
### 3.5 View the results
Run the following cell and when the table is shown, choose the <b>Bar</b> button to interactively visualize the data on a graph bar. Change the value on the <b>Y</b> dropdown list to select the yes, no and abstain votes

In [41]:
%%sql
select * from totals

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<br><br>
<a id="section4"></a>
## 4. Writing out results: partitioning

### 4.1 Write results
Let's start by writing out our totals DataFrame usin the <b>write()</b> function

In [42]:
path = "s3://{}/{}/lab1/out-totals/".format(S3_BUCKETNAME, S3_PREFIX)
totals.write.format("csv").option("header","true").save(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br><br>
### 4.2 Partitions
Now, let's find out how many partitions the DataFrame has using the <b>getNumPartitions()</b> function

In [43]:
totals.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3

If you open the <a href="https://console.aws.amazon.com/s3/buckets/">Amazon S3 Console</a>, choose your bucket and prefix for the lab, and navigate to the <b>"lab1/out-totals/"</b> prefix, you will find three CSV files corresponding to your partitions. Run the cell bellow to build the URL for you

In [ ]:
print("https://console.aws.amazon.com/s3/buckets/{}?prefix={}/lab1/out-totals/".format(S3_BUCKETNAME, S3_PREFIX))

<br><br>
### 4.3 Repartition
Now, let's repartition the DataFrame using the <b>coalesce()</b> function and then we write the results to <b>"lab1/out-totalsRep/"</b> prefix

In [46]:
totals = totals.coalesce(1)
path = "s3://{}/{}/lab1/out-totalsRep/".format(S3_BUCKETNAME, S3_PREFIX)
totals.write.format("csv").option("header","true").save(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Open the <a href="https://console.aws.amazon.com/s3/buckets/">Amazon S3 Console</a> and make sure the <b>"lab1/out-totalsRep/"</b> prefix only has 1 CSV file. Run the cell bellow to build the URL for you

In [ ]:
print("https://console.aws.amazon.com/s3/buckets/{}?prefix={}/lab1/out-totalsRep/".format(S3_BUCKETNAME, S3_PREFIX))

<br><br>
### 4.4 Partitioning by Column
Now lets take the Amazon listings DataFrame we used before, and write it to <b>"lab1/out-partitioning"</b>. Notice the <b>partitionBy()<b> function

In [49]:
path = "s3://{}/{}/lab1/out-partioning/".format(S3_BUCKETNAME, S3_PREFIX)
df.write.partitionBy("author").format("json").save(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Open the <a href="https://console.aws.amazon.com/s3/buckets/">Amazon S3 Console</a> and make sure the <b>"lab1/out-partioning/"</b> prefix has 1 <i>folder<i> per author. Run the cell bellow to build the URL for you

In [ ]:
print("https://console.aws.amazon.com/s3/buckets/{}?prefix={}/lab1/out-partioning/".format(S3_BUCKETNAME, S3_PREFIX))